# Covid-19: From model prediction to model predictive control

## Scenario-analysis with the age-stratified deterministic model

*Original code by Ryan S. McGee. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

Copyright (c) 2020 by T.W. Alleman, BIOMATH, Ghent University. All Rights Reserved.

This notebook was made to quickly perform scenario analysis with the age-stratified model implementation.

### Load required packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from ipywidgets import interact,fixed,FloatSlider,IntSlider,ToggleButtons
import pandas as pd
import datetime
import scipy
from scipy.integrate import odeint
import matplotlib.dates as mdates
import matplotlib
import scipy.stats as st
import networkx # to install networkx in your environment: conda install networkx
from covid19model.models import models
from covid19model.data import google
from covid19model.data import sciensano
from covid19model.data import polymod
from covid19model.data import parameters
from covid19model.visualization.output import population_status, infected
from covid19model.visualization.optimization import plot_fit
# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
import math
import xarray as xr

### Load interaction matrices

In [ ]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = polymod.get_interaction_matrices()
levels = initN.size

In [ ]:
params = parameters.get_COVID19_SEIRD_parameters()
params.update({'k': 1, 't0': 1})
initial_states = {'S': initN, 'E': np.ones(levels)}
model = models.COVID19_SEIRD(initial_states, params)

### Load parameter values, remove deterministic parameter, add compliance function parameters and intialise a stochastic model

In [ ]:
params = parameters.get_COVID19_SEIRD_parameters()
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#some required parameters are different, the 'parameter loading function' must be changed in the future
params.update({'beta':0.035,
               'k':   1,
               't0':5,
               'Nc': Nc_total})
del params['totalTests']
del params['dq']
del params['psi_FP']
del params['psi_PP']
initial_states = {'S': initN, 'E': np.ones(levels)}
model_sto = models.COVID19_SEIRD_sto(initial_states, params,discrete=True)

### Scrape high-level Sciensano data

In [ ]:
df_sciensano = sciensano.get_sciensano_COVID19_data(update=False)
df_sciensano.head()

### Initialise model

### Example simulation

In [ ]:
# Create checkpoints dictionary
chk = {'time':  [50], 
       'Nc':    [0.3*Nc_home],
      }
# Run a checkpointed simulation
out=model_sto.sim(120,checkpoints=chk)
# Visualise the number of pre-symptomatic infections over all ages
plt.plot(out["time"],out["I"].sum(dim="stratification"))
# Built-in function to visualise the number of patients in hospitals, in ICU and deaths
infected(out)

### Performing a calibration

Use the function fit_pso to obtain a good first estimate for the MCMC sampler.

In [ ]:
from covid19model.optimization import MCMC

# define dataset
data=[df_sciensano["H_in"].values[:7]]
# set optimisation settings
parNames = ['sigma_H_in','extraTime','beta'] # must be a list!
states = [["H_in"]]
bounds=((1,15),(32,48),(0.03,0.055)) # must be a list!
# run optimisation
theta = MCMC.fit_pso(model,data,parNames,states,bounds,maxiter=50,popsize=50)

In [ ]:
lag_time = int(round(theta[1]))
model.extraTime = int(round(theta[1]))

model.parameters.update({
    'beta': theta[2],
}
)

start_date = '15-03-2020'

In [ ]:
fig,ax=plt.subplots()
for i in range(100):
    y_model = model.sim(data[0].size+lag_time-1)
    ax = plot_fit(y_model,data,start_date,lag_time,states,ax=ax,plt_kwargs={'linewidth': 0.25,'alpha': 0.20})

Initiatize and run the sampler

In [ ]:
theta = [theta[0],theta[2]]
print(model.extraTime,theta)

In [ ]:
import emcee
from covid19model.optimization import objective_fcns

parNames = ['sigma_ICU','beta'] # must be a list!
states = [["ICU"]]
bounds=((1,100),(0.01,0.10))

pos = theta + [5, 1e-2 ]* np.random.randn(4, 2)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, objective_fcns.log_probability,
                                args=(model,bounds,data,states,parNames))
sampler.run_mcmc(pos, 2000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["sigma2_ICU","beta"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    #ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
sampler.get_autocorr_time()

In [ ]:
import corner

flat_samples = sampler.get_chain(discard=50, thin=1, flat=True)
fig = corner.corner(
    flat_samples, labels=labels,
);

In [ ]:
# Create checkpoints dictionary
chk = {'time':  [lag_time], 
       'Nc':    [0.2*Nc_home + 0.2*((1-0.70)*Nc_work + (1-0.80)*Nc_transport)],
      }

# define dataset
data=[df_sciensano["H_in"].values[:20]]
# set optimisation settings
parNames = ['sigma_H_in','k','t0'] # must be a list!
states = [["H_in"]]
bounds=((1,100),(0.2,1),(2,20)) # must be a list!
# run optimisation
theta = MCMC.fit_pso(model,data,parNames,states,bounds,checkpoints=chk,maxiter=100,popsize=1000)

In [ ]:
model.parameters.update({
    'k': theta[1],
    't0': theta[2]
}
)

In [ ]:
fig,ax=plt.subplots()
for i in range(2000):
    y_model = model.sim(data[0].size+lag_time-1,checkpoints=chk)
    ax = plot_fit(y_model,data,start_date,lag_time,states,ax=ax,plt_kwargs={'linewidth': 0.25,'alpha': 0.05})

# Notebook works till here

### Calibration of the hospitalization parameters

#### Calibrate $d_c$, $d_{icu}$ and $d_{icu, rec}$ to the new hospitalizations and hospital discharges

In [ ]:
tracedict = {
    'beta': np.asarray(trace['beta']),
    'dc': np.asarray(trace['dc']),
    'dICU': np.asarray(trace['dICU']),
    'dICUrec': np.asarray(trace['dICUrec'])
}

# save a copy in the raw folder
abs_dir = os.path.dirname(__file__)
rel_dir = os.path.join(abs_dir, '../../data/interim/model_parameters/trace.csv')

(pd.DataFrame.from_dict(data=tracedict, orient='index')
   .to_csv(reldir, header=False))

In [ ]:
with COVID19MODEL:
    # draw 1000 posterior samples
    trace = pm.sample(10,start=map_estimate,step=pm.Slice(),cores=1)
pm.traceplot(trace,varnames=['dc','dICU','dICUrec'])

In [ ]:
tracedict_others = {
    'dc': np.asarray(trace['dc']),
    'dICU': np.asarray(trace['dICU']),
    'dICUrec': np.asarray(trace['dICUrec'])
}

### Scenarios


Use the code snippet below to see the correspondence between `'t'` in the `pastPolicy` dictionary and the actual date.

In [ ]:
# Define data as a list containing data timeseries
data=[np.transpose(ICU_tot),np.transpose(H_tot)]
print(index[2],index[50],index[57],index[-1])

The interaction matrices of the 2008 study by Mossong were gathered under a business-as-usual scenario. It is thus not possible to use the interaction matrices without doing a correction for social distancing. Even when using only the interactions at home (`Nc_home`), the virus cannot be stopped.

In [ ]:
sigma = 1

Stringent measures were taken in the evening of March 17th, which corresponds to time-index 3, however, the measures are only 'enforced' on day 8 in the `pastPolicy` dictionary. Why? The change in human behaviour was very gradual, it took between 10-14 days after March 17th before everyone was working at home (see the Google mobility report). In other words, measures were taken on March 17th, but obedience for these measures was gradual, like a ramp. However, in the model we make a step-wise change. The obedience to measures can be adressed in future work.

Another important issue to adress is the home interaction matrix `Nc_home`. All contacts in these matrices are still assumed to be random, during a lockdown, the interactions at home should somehow be corrected for the 'bubble' effect. Since the average household size in belgium is 2 people, I correct the `Nc_home` matrix with a factor 1/2.

In [ ]:
sigma = 0.6

# Create a dictionary of past policies
pastPolicy = {'t':       [1,50,64], 
              'Nc':      [0.3*Nc_home+sigma*((1-0.70)*Nc_work+(1-0.70)*Nc_transport),
                          0.3*Nc_home+sigma*((1-0.30)*Nc_work+(1-0.40)*Nc_transport+(1-0.75)*Nc_schools),
                          0.3*Nc_home+sigma*((1-0.30)*Nc_work+(1-0.40)*Nc_transport+Nc_schools),
                         ]
              }

In [ ]:
# Create a dictionary of future policies
futurePolicy = {'t':     [21], # May 21th, June 4th 
                'Nc':    [0.3*Nc_home+sigma*((1-0.30)*Nc_work+(1-0.40)*Nc_transport)],
              }

In [ ]:
positions=[np.array([6]),np.array([5,6])]
# Run realTimeScenario
model.realTimeScenario(startdate,data,positions,pastPolicy,futurePolicy=futurePolicy,trace={'beta': [model.beta]},T_extra=124,
                       modelClr=['red','orange'],legendText=('ICU (model)','Hospital (model)','ICU (data)','Hospital (data)'),
                       titleText='Belgium',filename='test.svg')

Save your figures by altering the variable `filename = xxxxx.svg`!